Importing Libraries

In [1]:
!pip uninstall -y transformers trl peft accelerate bitsandbytes

!pip install "transformers>=4.40.0"
!pip install "trl==0.9.4"
!pip install "peft>=0.11.1"
!pip install "accelerate>=0.29.2"
!pip install bitsandbytes


Found existing installation: transformers 4.57.2
Uninstalling transformers-4.57.2:
  Successfully uninstalled transformers-4.57.2
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.0 MB/s eta 0:00:00


Uploading Train and Test Excel file

In [2]:
from google.colab import files

uploaded = files.upload()


Saving train-test-data.xlsx to train-test-data.xlsx


Importing to test and train dataframes

In [3]:
import pandas as pd

file_path = "train-test-data.xlsx"
train_df = pd.read_excel(file_path, sheet_name="train")
test_df  = pd.read_excel(file_path, sheet_name="test")

print("Train dataset sample:")
display(train_df.head())

print("Test dataset sample:")
display(test_df.head())

Train dataset sample:


,case_id,input_finding,output_disease
0,1,The liver surface is regular with no apparent ...,Liver lesion
1,2,"The liver and spleen are in normal position, s...","Kidney stone, Pancreatitis"
2,3,"The liver surface is smooth, and the size and ...","Liver cyst, Renal cyst, Splenic cyst"
3,4,"Abnormal liver morphology, with arc-shaped low...","Liver lesion, Liver cyst, Gallstone, Ascites, ..."
4,5,"Two round low-density lesions, approximately 5...","Liver lesion, Splenomegaly, Renal mass, Colore..."


Test dataset sample:


,case_id,input_finding,output_disease
0,1,The liver is normal in size and shape with hom...,"Renal cyst, Adrenal hyperplasia, Adrenal calci..."
1,2,"The liver is normal in position, size, and sha...","Liver lesion, Atherosclerosis"
2,3,"The liver surface is smooth, with coordinated ...","Liver cyst, Pancreas calcifications, Accessory..."
3,4,The liver surface is smooth with coordinated s...,"Cholecystitis, Renal cyst, Ascites"
4,5,"The surface of the liver is smooth, and the si...","Liver calcifications, Gallstone"


Importing the Qwen3-8B model in 4-bit

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Filtering all the possible outputs of the diseases since it is a multiclass output

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
df = pd.read_excel("train-test-data.xlsx")
label_lists = [list(dict.fromkeys(lbl.split(", "))) for lbl in df['output_disease']]

mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(label_lists)

classes = mlb.classes_
print("All disease classes:", classes)
print("Multi-label encoded shape:", labels_encoded.shape)

All disease classes: ['Abdominal abscess' 'Abdominal aortic aneurysm' 'Abdominal lipiditis'
 'Abdominal mass' 'Absent kidney' 'Accessory spleen'
 'Adrenal calcification' 'Adrenal hyperplasia' 'Adrenal mass'
 'Adrenal metastasis' 'Adrenal nodule' 'Annular pancreas' 'Appendicitis'
 'Arteriosclerosis' 'Ascites' 'Atherosclerosis' 'Basal cell carcinoma'
 'Bile duct cancer' 'Bile duct carcinoma' 'Bile duct dilatation'
 'Bile duct stone' 'Cecal epiploic appendagitis' 'Cholecystitis'
 'Cirrhosis' 'Colitis' 'Colorectal cancer' 'Diverticulitis' 'Enteritis'
 'Enteritis possible' 'Epiploic appendagitis' 'Fatty liver'
 'Gallbladder mass' 'Gallstone' 'Gastric cancer' 'Gastric lymphoma'
 'Gastrointestinal bleed' 'Gastrointestinal perforation' 'Hepatitis'
 'Hernia' 'Hydronephrosis' 'Inflammatory bowel disease'
 'Intestinal obstruction' 'Kidney infarction' 'Kidney stone'
 'Liver abscess' 'Liver calcification' 'Liver cyst' 'Liver lesion'
 'Lymphadenopathy' 'Neuroblastoma' 'No acute abnormality'
 'Omenta

Prompt template for the model

In [10]:
prompt2 = """You are a strict medical label extraction system.

TASK:
Extract ONLY the disease labels explicitly from the known diseases classes. NO EXPLANATIONS.
NO EXTRA TEXT. JUST COMMA-SEPARATED LIST.

Known disease classes: {classes}
STRICT RULES:
- Output ONLY a comma-separated list of disease labels.
- Do NOT explain.
- Do NOT describe your reasoning.
- Do NOT rewrite the findings.
- Do NOT introduce new diseases.
- If multiple diseases appear, separate them with commas.
- If none are found, output: None
- Output NOTHING outside the tag.


VALID OUTPUT FORMAT:
<disease1>, <disease2>, <disease3>

Finding:
{finding}

Output:"""

Predicting the possible outut based on the description of the disease

In [11]:
def predict_baseline(finding, classes):
    prompt = prompt2.format(finding=finding, classes = classes)
    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **tokens,
        max_new_tokens=30,
        temperature=0.0,
        do_sample=False
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.split("Output:")[-1].strip()


Testing a case

In [12]:
finding = """The liver is normal in size and shape with homogeneous density. A patchy low-density
lesion is seen around the liver ssure. The intrahepatic duct system is not obviously
dilated, and the course is normal. The gallbladder is not enlarged, with no obvious
thickening of the wall, and no clearly abnormal density foci are seen inside. The spleen is
normal in size and shape with homogeneous density, and some punctate low-density foci
are seen inside. The pancreas is normal in size and shape with homogeneous density, and
no clearly abnormal density foci are seen inside. The main pancreatic duct is not obviously
dilated, and the peripancreatic fat space is clear. Both kidneys are normal in size and shape
with homogeneous density. A round low-density lesion is seen in the right kidney with a
diameter of about 16mm. The left adrenal gland is thickened, and a punctate high-density
lesion is seen in the right adrenal gland. The renal pelvis-calyx system is not obviously
dilated. The perirenal fat space is clear, and no clearly abnormal density foci are seen. No
enlarged lymph nodes are seen in the retroperitoneum.
"""
print(predict_baseline(finding, classes))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<Abdominal abscess>, <Abdominal aortic aneurysm>, <Abdominal lipiditis>, <Abdominal


In [13]:
test_df

,case_id,input_finding,output_disease
0,1,The liver is normal in size and shape with hom...,"Renal cyst, Adrenal hyperplasia, Adrenal calci..."
1,2,"The liver is normal in position, size, and sha...","Liver lesion, Atherosclerosis"
2,3,"The liver surface is smooth, with coordinated ...","Liver cyst, Pancreas calcifications, Accessory..."
3,4,The liver surface is smooth with coordinated s...,"Cholecystitis, Renal cyst, Ascites"
4,5,"The surface of the liver is smooth, and the si...","Liver calcifications, Gallstone"
...,...,...,...
381,382,"The appendix is swollen and thickened, with no...","Appendicitis, Peritonitis, Pneumoperitoneum"
382,383,The appendix is significantly thickened with i...,"Appendicitis, Gallstone, Cholecystitis, Renal ..."
383,384,Mild dilation of the abdominal intestinal cana...,"Intestinal obstruction, Kidney stone, Atherosc..."
384,385,"The liver shape, size, and proportions of indi...","Liver cyst, Renal cyst, Gallstone"


Finding predictions on Qwen3 8b base model on test dataframe

In [14]:
predictions = []

for finding in test_df["input_finding"]:
    labels = predict_baseline(finding, classes)
    predictions.append(labels)
    print("done")

test_df["predicted_labels"] = predictions
test_df.head()


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


,case_id,input_finding,output_disease,predicted_labels
0,1,The liver is normal in size and shape with hom...,"Renal cyst, Adrenal hyperplasia, Adrenal calci...","<disease1>, <disease2>, <disease3>\nBased on t..."
1,2,"The liver is normal in position, size, and sha...","Liver lesion, Atherosclerosis","<Abdominal mass>,<Pancreatic mass>,<Splenomega..."
2,3,"The liver surface is smooth, with coordinated ...","Liver cyst, Pancreas calcifications, Accessory...","<Abdominal mass>,<Ascites>,<Bile duct stone>,<..."
3,4,The liver surface is smooth with coordinated s...,"Cholecystitis, Renal cyst, Ascites","None\nOkay, let's see. I need to extract the d..."
4,5,"The surface of the liver is smooth, and the si...","Liver calcifications, Gallstone","<calcification>, <calcified nodule>\nOkay, let..."


Evaluation scores based on AJccard, Micro F1 and Macro F1

In [15]:
def to_list(x):
    if isinstance(x, list):
        return x
    return [i.strip() for i in x.split(",") if i.strip()]

test_df["gt_labels"] = test_df["output_disease"].apply(to_list)
test_df["pred_labels"] = test_df["predicted_labels"].apply(lambda x: x if isinstance(x, list) else to_list(x))


In [16]:
def jaccard(a, b):
    a, b = set(a), set(b)
    if len(a | b) == 0:
        return 1.0
    return len(a & b) / len(a | b)

test_df['jaccard'] = test_df.apply(lambda row: jaccard(row['gt_labels'], row['pred_labels']), axis=1)


In [17]:
overall_jaccard = test_df['jaccard'].mean()
overall_jaccard


np.float64(0.01759586124871099)

In [18]:
from sklearn.metrics import f1_score


label_list = classes

def to_binary_vector(labels):
    return [1 if l in labels else 0 for l in label_list]

y_true = test_df["gt_labels"].apply(to_binary_vector).tolist()
y_pred = test_df["pred_labels"].apply(to_binary_vector).tolist()

micro_f1 = f1_score(y_true, y_pred, average="micro")
macro_f1 = f1_score(y_true, y_pred, average="macro")

micro_f1, macro_f1


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.07756232686980609, 0.03999933088070834)

In [19]:
print("Overall Jaccard:", overall_jaccard)
print("Micro F1:", micro_f1)
print("Macro F1:", macro_f1)

Overall Jaccard: 0.01759586124871099
Micro F1: 0.07756232686980609
Macro F1: 0.03999933088070834


In [20]:
test_df.to_excel("model_predictions_with_scores_base8b.xlsx", index=False)
